<a href="https://colab.research.google.com/github/hkbharath/BazzingaAI/blob/main/Expectimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Used the code from
# Copyright 2014 Google Inc. All rights reserved.

import random
import sys
import time
import math

# board size
N = 4

class Moves:
  MOVE_LEFT = 'left'
  MOVE_UP = 'up'
  MOVE_RIGHT = 'right'
  MOVE_DOWN = 'down'

class GameTracker:
  def __init__(self):
    self.no_moves = 0
    self.st_time = time.time()
    self.end_time = 0
    self.max_tile = 2
    self.score = 0

  def getTimePerMove(self):
    return (self.end_time - self.st_time)/self.no_moves

  def getMaxTile(self):
    return self.max_tile
  
  def getNoOfMoves(self):
    return self.no_moves

  def getScore(self):
    return self.score

class Board(object):
  def __init__(self, grid=None):
    
    self.over = False
    self.gt = GameTracker()
    self.enableRandomTile = True

    if grid:
      self.board = grid
      self.is_custom_board = True 
      # check if the game is already over
      if len(self.get_next_moves()) == 0:
        self.over = True 
    else:
      self.board = [[None] * N for i in range(N)]
      self.is_custom_board = False
      self.randomTile()
      self.randomTile()

  #Would be needed for Deep RL
  def startGame():
    if self.gt.score == 0 and not self.over:
      self.gt = GameTracker()

  #Would be needed for Deep RL
  def startGame():
    if self.gt.score == 0 and not self.over:
      self.gt = GameTracker()

  def rotateLeft(self, grid):
    out = self.emptyGrid()
    for c in range(N):
      for r in range(N):
        out[r][N-1-c] = grid[c][r]
    return out

  def rotateRight(self, grid):
    out = self.emptyGrid()
    for c in range(N):
      for r in range(N):
        out[N-1-r][c] = grid[c][r]
    return out

  def emptyGrid(self):
    out = list()
    for x in range(N):
      col = list()
      for y in range(N):
        col.append(None)
      out.append(col)
    return out

  def to_move(self, grid, direction):
    
    out = self.emptyGrid()

    if direction == Moves.MOVE_UP:
      rot = 1
    elif direction == Moves.MOVE_RIGHT:
      rot = 2
    elif direction == Moves.MOVE_DOWN:
      rot = 3
    else:
      rot = 0

    for i in range(rot):
      grid = self.rotateLeft(grid)

    score = 0
    for r in range(N):
      oc = 0
      ic = 0
      while ic < N:
        if grid[ic][r] is None:
          ic += 1
          continue
        out[oc][r] = grid[ic][r]
        oc += 1
        ic += 1

      ic = 0
      oc = 0
      while ic < N:
        if out[ic][r] is None:
          break
        if ic == N-1:
          out[oc][r] = out[ic][r]
          oc += 1
          break
        if out[ic][r] == out[ic+1][r]:
          #out[oc][r] *= 2
          out[oc][r] = 2*out[ic][r]
          score += out[oc][r]
          if self.gt.max_tile < out[oc][r]:
            self.gt.max_tile = out[oc][r]
          ic += 1
        else:
          out[oc][r] = out[ic][r]
        ic += 1
        oc += 1
      while oc < N:
        out[oc][r] = None
        oc += 1

    for i in range(rot):
      out = self.rotateRight(out)

    return out, score

  def move(self, direction):
    #print 'move', direction
    next_board, got_score = self.to_move(self.board, direction)
    moved = (next_board != self.board)

    self.board = next_board
    self.gt.score += got_score

    if moved:
      self.gt.no_moves = self.gt.no_moves + 1
      if not self.randomTile() or len(self.get_next_moves()) == 0:
        self.over = True
        self.gt.end_time = time.time()

  def canMove(self, direction, grid=None):
    if not grid:
      grid = self.board
    return self.board != self.to_move(grid, direction)[0]

  def get_empty_cells(self):
    for i in range(N):
      for j in range(N):
        if self.board[i][j] is None:
          yield i, j

  def get_next_moves(self):
    moves = []
    if self.over:
      return moves
    for move in [Moves.MOVE_LEFT, Moves.MOVE_DOWN, Moves.MOVE_RIGHT, Moves.MOVE_UP]:
      if self.canMove(move):
        moves.append(move)
    return moves

  def randomTile(self):
    if not self.enableRandomTile:
      return True
    cells = list(self.get_empty_cells())
    if not cells:
      return False
    #print 'cells', cells


    if random.random() < 0.9:
      v = 2
    else:
      v = 4

    cid = random.choice(cells)
    #print cid
    self.board[cid[0]][cid[1]] = v
    return True

  def enableRandomTile(rtile):
    self.enableRandomTile = rtile

  def show(self):
    for i in range(N):
      print ('|', end='')
      for j in range(N):
        if self.board[j][i]:
          print (' %4d |' % self.board[j][i], end='')
        else:
          print ('    . |', end='')
      print("")

class GameManager():
  def __init__(self, grid = None):
    self.board = Board(grid)

  def getCurrentState(self):
    return self.board.board

  def getScore(self):
    return self.board.gt.score

  def isOver(self):
    return self.board.over

  def getAvailableMoves(self):
    return self.board.get_next_moves()
  
  def makeMove(self, direction):
    if self.isOver() or not self.board.canMove(direction):
      return False
    self.board.move(direction)
    return True
  
  def tryMove(self, grid, direction):
    if not self.board.canMove(direction, grid=grid):
      return grid, 0
    return self.board.to_move(grid, direction)
    
  def printState(self):
    self.board.show()

  def getGameTracker(self):
    return self.board.gt

  def getNoOfEmptyCells(self):
    return len(list(self.board.get_empty_cells()))
  
  def getEmptyCells(self):
    return list(self.board.get_empty_cells())
  
  def evalSmoothness(self, grid=None):
    if grid is None:
      grid = self.board.board
    score_smooth = 0
    N = len(grid)
    for x in range(N-1):
        for y in range(N-1):
            s = 0
            # s += abs((grid[x][y] or 2) - (grid[x+1][y] or 2))
            # s += abs((grid[x][y] or 2) - (grid[x][y+1] or 2))
            if grid[x][y] and grid[x+1][y]:
              s += abs((math.log(grid[x][y])/math.log(2)) - (math.log(grid[x+1][y])/math.log(2)))
            if grid[x][y] and grid[x][y+1]:
              s += abs((math.log(grid[x][y])/math.log(2)) - (math.log(grid[x][y+1])/math.log(2)))
            score_smooth -= s
    return score_smooth
  
  def evalFreeCells(self, grid=None):
    if grid is None:
      grid = self.board.board

    empty_count = 0

    for row in grid:
      # one of them will be there
      empty_count += row.count(None)
      empty_count += row.count(0)
    
    # print(empty_count)
    # print(grid)
    # return (16-empty_count)**2
    if empty_count == 0:
      return 0
    return math.log(empty_count)
  
  def evalMaxTile(self, grid=None):
    if grid is None:
      grid = self.board.board
    
    return max([0 if tile is None else tile for row in grid for tile in row])


  def evalMonotone(self, grid=None):
    if grid is None:
      grid = self.board.board
    
    for x in range(N):
      for y in range(N):
        if grid[x][y] is None:
          grid[x][y] = 0

    L = R = U = D = 0
    LR = UD = 0
    for x in range(N):
      m = 0
      for y in range(N-1):
        if grid[x][y] and grid[x][y] >= grid[x][y+1]:
          m += 1
          L += m ** 2 * 4
        else:
          L -= abs((grid[x][y] or 0)- (grid[x][y+1] or 0)) * m ** 2
          m = 0

      m = 0
      for y in range(N-1):
        if grid[x][y] <= grid[x][y+1] and grid[x][y+1]:
          m += 1
          R += m ** 2 * 4
        else:
          R -= abs((grid[x][y] or 0)- (grid[x][y+1] or 0)) * m ** 2
          m = 0

    LR += min(L, R)
    L = R = 0

    for y in range(N):
      m = 0
      for x in range(N-1):
        if grid[x][y] and grid[x][y] >= grid[x+1][y]:
          m += 1
        else:
          #U -= abs(to_idx[grid[x][y]] - to_idx[grid[x+1][y]]) ** 2
          U -= abs((grid[x][y] or 0)- (grid[x+1][y] or 0)) * m ** 2
          m = 0

      m = 0
      for x in range(N-1):
        if grid[x][y] <= grid[x+1][y] and grid[x+1][y]:
          m += 1
          D += m ** 2 * 4
        else:
          D -= abs((grid[x][y] or 0)- (grid[x+1][y] or 0)) * m ** 2
          m = 0

    UD += min(U, D)

    return LR + UD

  def evalMonotone_simple(self, grid=None):
    if grid is None:
      grid = self.board.board
    
    for x in range(N):
      for y in range(N):
        if grid[x][y] is None:
          grid[x][y] = 0

    L = R = U = D = 0
    LR = UD = 0
    for x in range(N):
      for y in range(N-1):
        cval = math.log(grid[x][y])/math.log(2) if grid[x][y] else 0
        nval = math.log(grid[x][y+1])/math.log(2) if grid[x][y+1] else 0
        if cval > nval:
          L +=  nval - cval
        else:
          R += cval - nval

    LR = max(L, R)
    L = R = 0

    for y in range(N):
      for x in range(N-1):
        cval = math.log(grid[x][y])/math.log(2) if grid[x][y] else 0
        nval = math.log(grid[x+1][y])/math.log(2) if grid[x+1][y] else 0
        if cval > nval:
          U += nval - cval
        else:
          D += cval - nval
      
    UD = max(U, D)

    return (LR + UD)

In [ ]:
# Simulation to test the game 
# This makes fully random moves till the game ends


import random
from statistics import mean, variance, stdev, mode
import time
import sys

T=2
DEBUG = False

class ExpectimaxAgent():
    def __init__(self, N=4):
        self.scores = {}
        self.N = N
        self.depth = 2

    def encode(self, grid):
        key = ','.join([str(tile) for row in grid for tile in row])
        return key

    def simulation_random_move(self, sim_gm, depth, h_score):
        tot_score = 0

        if sim_gm.isOver():
            return tot_score

        empty_cell_list = sim_gm.getEmptyCells()
        pcell = 1/len(empty_cell_list)

        for pos in empty_cell_list:
            #Try assigning 4 in current empty slot
            grid = sim_gm.getCurrentState()
            grid[pos[0]][pos[1]] = 4
            tot_score = tot_score + 0.1*pcell*self.expectimax(grid, depth-1, False, h_score)
            grid[pos[0]][pos[1]] = 0

            grid[pos[0]][pos[1]] = 2
            tot_score = tot_score + 0.9*pcell*self.expectimax(grid, depth-1, False, h_score)
            grid[pos[0]][pos[1]] = 0
        return tot_score

    def heuristic(self, sim_gm, h_score):
        #set using trail and error
        w_freecell = 2.5
        w_monotone = 0.75
        w_maxtile = 1
        w_smooth = 0.5

        key = self.encode(sim_gm.getCurrentState())
        if key in self.scores:
            return self.scores[key]
        final_score = 0 #h_score
        final_score += w_freecell * sim_gm.evalFreeCells() 
        final_score += w_smooth * sim_gm.evalSmoothness()
        final_score += w_monotone*sim_gm.evalMonotone_simple()
        final_score += w_maxtile * sim_gm.evalMaxTile()
        # final_score += sim_gm.evalMonotone()

        if random.randint(0,100) == 0 and DEBUG == True:
            sim_gm.printState()
            print("Free Cell %.3f"%(sim_gm.evalFreeCells()))
            print("Smoothness %.3f"%(sim_gm.evalSmoothness()))
            print("Monotone Simple %.3f"%(sim_gm.evalMonotone_simple()))
            print("Max Tile %d"%(sim_gm.evalMaxTile()))
            print("Score %d"%(h_score))

        self.scores[key] = final_score
        return self.scores[key] 

    def expectimax(self, grid, depth, is_random, h_score):
        sim_gm = GameManager(grid)
        # sim_gm.enableRandomTile(False) # May not be needed
        if depth == 0 or sim_gm.isOver():
            return self.heuristic(sim_gm, h_score)
        elif is_random:
            return self.simulation_random_move(sim_gm, depth, h_score)    
        else:
            # our move
            max_score = float('-inf')
            moves = sim_gm.getAvailableMoves()
            for game_move in moves:
                current_grid, current_score = sim_gm.tryMove(grid, game_move)
                max_score = max(max_score, self.expectimax(current_grid, depth, True, h_score + current_score))
            return max_score
        return -1
    
    def getNextMove(self, grid, moves):
        best_move = moves[0]
        best_sc = float('-inf')
        sim_gm = GameManager(grid)

        for move in moves:
            current_grid, current_score = sim_gm.tryMove(grid, move)
            depth = self.depth
            if sim_gm.getNoOfEmptyCells() <= 2:
                depth += 1 
            current_score = self.expectimax(current_grid, depth, True, current_score)
            # print("Score %.3f:%s"%(current_score, move))
            if current_score > best_sc:
                best_move = move
                best_sc = current_score
        # print("")
        return best_move

def run():
    game = GameManager()
    gt = game.getGameTracker()

    ai = ExpectimaxAgent(4) # 4x4 tile game
    while not game.isOver():
    
        moves = game.getAvailableMoves()
        # random.shuffle(moves)
        next_move = ai.getNextMove(game.getCurrentState(), moves)
        game.makeMove(next_move)
        # print("Move made: %s"%(next_move))
        # if not DEBUG:
        #     game.printState()
    game.printState()
    print("Final Score: %d"%(game.getScore()))
    print("Max Tile: %d"%(gt.getMaxTile()))
    print("No. Of Moves: %d"%(gt.getNoOfMoves()))
    print("Time per  move: %0.5f"%(gt.getTimePerMove()))
    # print(ai.scores)
    return gt

if __name__ == "__main__":
    print("Expectimax Bot")
    t_scores = []
    ex_times = []
    max_tile = []
    num_moves = []
    while T > 0:
        gt = run()
        t_scores.append(gt.getScore())
        ex_times.append(gt.getTimePerMove())
        max_tile.append(gt.getMaxTile())
        num_moves.append(gt.getNoOfMoves())
        T = T-1
    print("Final Analysis:")
    print("Mean Score: %.3f"%(mean(t_scores)))
    print("Max Score: %d"%(max(t_scores)))
    print("Std Deviation of Scores: %.3f"%(stdev(t_scores)))
    print("Average Time taken per move(sec): %.4f"%(mean(ex_times)))
    print("Average No. of moves: %.4f"%(mean(num_moves)))
    print("Most common Max Tile reached: %d"%(mode(max_tile)))

Expectimax Bot
|    2 |    4 |   16 |    8 |
|    4 |   16 |   32 |   64 |
|    8 |   64 |  128 |   16 |
|   16 |   32 |   64 |  512 |
Final Score: 6132
Max Tile: 512
No. Of Moves: 446
Time per  move: 0.46790
|    8 |    2 |    4 |    2 |
|   16 |   32 |   16 |    8 |
|   32 |  128 | 2048 |   16 |
|  128 |  256 | 1024 |   32 |
Final Score: 32952
Max Tile: 2048
No. Of Moves: 1715
Time per  move: 0.42722
Final Analysis:
Mean Score: 19542.000
Max Score: 32952
Std Deviation of Scores: 18964.604
Average Time taken per move(sec): 0.4476
Average No. of moves: 1080.5000


StatisticsError: ignored